In [16]:
import numpy as np
import pandas as pd

In [17]:
import calendar
import time

def getCurrentTimestamp():
    return int(calendar.timegm(time.gmtime()))

from datetime import datetime

#for ex: 06/12/2021  06:55:00
def convert_date_2_timestamp(time_str):
    #print(time_str)
    try:
        dt = datetime.strptime(time_str, "%d/%m/%Y %H:%M")
        return dt.timestamp()
    except ValueError:
        print("Invalid time format. Please use 'dd/mm/yyyy HH:MM:SS'")
    return None

time_str = "18/2/2024 06:55"
timestamp = convert_date_2_timestamp(time_str)
print(f"Timestamp for {time_str}: {timestamp}") 

Timestamp for 18/2/2024 06:55: 1708239300.0


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, recall_score, explained_variance_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 

import matplotlib.pyplot as plt
import seaborn as sns  # For nicer plots (optional)

import joblib  # or import pickle


In [19]:
# Load the model later
loaded_model = joblib.load('random_forest_model_1.joblib')  # or pickle.load(open('random_forest_model.pkl', 'rb'))

# Use the loaded model for predictions
predictions = loaded_model.predict(pd.DataFrame({
                    'from_turbine': [1], 
                    'to_turbine': [2], 
                    'from_wave_h': [1.304],
                    'from_e_wind': [0.95]	, 
                    'from_n_wind': [8.94], 
                    'from_e_current': [-0.065], 
                    'from_n_current': [-0.305], 
                    'from_time': [1638781000]
                }))
print(predictions)

[228.49270392]


In [20]:
points = [0, 2, 3, 5, 7]
from_time = 1638781000

#break down into pairs of turbines
import itertools
pairs = list(itertools.combinations(points, 2))  # 2 specifies pairs
print(pairs)
pair_len = len(pairs)
distances = {}   #key: pair, value: predicted fuel
for i in range(0, pair_len):
    predicted_value = loaded_model.predict(pd.DataFrame({
                    'from_turbine': [pairs[i][0]], 
                    'to_turbine': [pairs[i][1]], 
                    'from_wave_h': [1.304],
                    'from_e_wind': [0.95]	, 
                    'from_n_wind': [8.94], 
                    'from_e_current': [-0.065], 
                    'from_n_current': [-0.305], 
                    'from_time': [1638781000]
                }))
    distances[pairs[i]] = predicted_value[0]

[(0, 2), (0, 3), (0, 5), (0, 7), (2, 3), (2, 5), (2, 7), (3, 5), (3, 7), (5, 7)]


In [21]:
distances

{(0, 2): 853.2833940330391,
 (0, 3): 849.1900255468504,
 (0, 5): 847.1056140089172,
 (0, 7): 849.5357101309284,
 (2, 3): 267.5227924683428,
 (2, 5): 281.4817788456153,
 (2, 7): 286.7248997253633,
 (3, 5): 282.031871795202,
 (3, 7): 287.27499267494994,
 (5, 7): 309.1277734903459}

In [22]:
def shortest_path_starting_from_first(points, distances):
    """
    Finds the shortest path that visits all points exactly once, 
    starting from the first point in the list.

    Args:
        points: A list of point labels (e.g., ['A', 'B', 'C', 'D', 'E']).
        distances: A dictionary where keys are tuples of two points (in any order)
                   and values are the distances between them.

    Returns:
        A tuple containing:
            - The shortest path (a list of point labels in order).
            - The total distance of the shortest path.
    """

    if not points:
        return [], 0  # Handle empty points list

    start_point = points[0]
    remaining_points = points[1:]

    min_distance = float('inf')
    shortest_path_found = None

    for path_permutation in itertools.permutations(remaining_points):
        current_path = [start_point] + list(path_permutation) # Prepend the starting point
        total_distance = 0
        for i in range(len(current_path) - 1):
            current_point = current_path[i]
            next_point = current_path[i + 1]
            distance = distances.get((current_point, next_point)) or distances.get((next_point, current_point))
            if distance is None:
                raise ValueError(f"Distance between {current_point} and {next_point} not defined.")
            total_distance += distance

        if total_distance < min_distance:
            min_distance = total_distance
            shortest_path_found = current_path

    return shortest_path_found, min_distance


# Example usage:
#points = ['A', 'B', 'C', 'D', 'E']

#distances = {
#    ('A', 'B'): 10, ('B', 'A'): 10,
#    ('A', 'C'): 15, ('C', 'A'): 15,
#    ('A', 'D'): 20, ('D', 'A'): 20,
#    ('A', 'E'): 25, ('E', 'A'): 25,
#    ('B', 'C'): 35, ('C', 'B'): 35,
#    ('B', 'D'): 25, ('D', 'B'): 25,
#    ('B', 'E'): 30, ('E', 'B'): 30,
#    ('C', 'D'): 15, ('D', 'C'): 15,
#    ('C', 'E'): 20, ('E', 'C'): 20,
#    ('D', 'E'): 10, ('E', 'D'): 10,
#}

shortest_path_result, min_distance_result = shortest_path_starting_from_first(points, distances)
print("Shortest Path:", shortest_path_result)
print("Total Fuel:", min_distance_result)



Shortest Path: [0, 5, 2, 3, 7]
Total Fuel: 1683.3851779978252
